<a href="http://landlab.github.io"><img style="float: left" src="../../landlab_header.png"></a>

# Landscape evolution model with Priority flood and Space_v2
<hr>

The priority flood flow director is designed to calculate flow properties over large scale grids. In the following notebook we illustrate how the priority flood flow accumulator can be used to simulate landscape evolution using the SPAVE_V2 Landlab component

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

from landlab import RasterModelGrid, imshow_grid
from landlab.components import PriorityFloodFlowRouter, SpaceLargeScaleEroder

Create raster grid

In [ ]:
mg = RasterModelGrid((25, 25), xy_spacing=10.0)
z = mg.add_zeros("topographic__elevation", at="node")
mg.at_node["topographic__elevation"][mg.core_nodes] += np.random.rand(
    mg.number_of_core_nodes
)

s = mg.add_zeros("soil__depth", at="node", dtype=float)
mg.at_node["soil__depth"][mg.core_nodes] += 0.5
mg.at_node["topographic__elevation"] += mg.at_node["soil__depth"]

# fr = FlowAccumulator(mg, flow_director='D8')
# df = DepressionFinderAndRouter(mg)

fr = PriorityFloodFlowRouter(mg, flow_metric="D8", update_flow_depressions=True)


ha = SpaceLargeScaleEroder(mg, K_sed=0.0001, K_br=0.0001, phi=0.3, H_star=1)

br = mg.at_node["bedrock__elevation"]
z = mg.at_node["topographic__elevation"]

space_dt = 250

In [ ]:
z_ori = np.array(z)
for i in tqdm(range(250)):
    # Uplift
    br[mg.core_nodes] += 0.001 * space_dt
    z[mg.core_nodes] = br[mg.core_nodes] + s[mg.core_nodes]
    fr.run_one_step()
    # df.map_depressions()
    ha.run_one_step(dt=space_dt)

In [ ]:
imshow_grid(mg, "topographic__elevation", cmap="terrain")
plt.title("Final topographic__elevation")